# Import tools

In [ ]:
!pip install gingerit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import os
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import spacy
from gingerit.gingerit import GingerIt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Reading the text files

In [ ]:
txt=[]
from google.colab import files
uploaded_files = files.upload()

for file in uploaded_files.keys():
  with open(file) as f:
    for line in f:
      line = line.strip()
      if line=='----------': break
      if line!='':txt.append(line)

print("number of lines = ", len(txt))



Saving 3gab.txt to 3gab (1).txt
Saving 3stu.txt to 3stu.txt
Saving abbe.txt to abbe (1).txt
Saving advs.txt to advs.txt
Saving blan.txt to blan.txt
Saving cnus.txt to cnus.txt
Saving danc.txt to danc.txt
Saving empt.txt to empt.txt
number of lines =  66550


# Cleaning the text

In [ ]:
def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt+=words
    return cleaned_txt

cleaned_txt = clean_txt(txt)
print("number of words = ", len(cleaned_txt))

number of words =  723502


## Training the model

In [ ]:
def make_markov_model(cleaned_stories, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_stories)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_stories[i+j] + " "
            next_state += cleaned_stories[i+j+n_gram] + " "
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1

    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total

    return markov_model

In [ ]:
markov_model = make_markov_model(cleaned_txt)
print("number of states = ", len(markov_model.keys()))
markov_model.keys()

# Generate Text

In [ ]:
def generate_story(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story += curr_state + " "
    while n < limit:
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))

        curr_state = next_state[0]
        story += curr_state + " "
        n += 1
    # story = add_punctuation(story)
    return story

In [ ]:
def grammaticize_text(text):
    ginger_parser = GingerIt()
    result = ginger_parser.parse(text)
    return result['result']

In [ ]:
def grammaticize_text1(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    # sentences=grammaticize_text(sentences)
    return ".".join(sentences)


In [ ]:
story=generate_story(markov_model, start="i felt", limit=30)
result=grammaticize_text1(story)
sent=grammaticize_text(result)
# gramm_story=sent.replace(".\n", ".")
print(str(i)+". ", sent)

0.  I felt the room, examining the window someone had brought up a will which he did split I didn't wait to see justice done you can remove your man if I am thinking of the gallantry which was a hundred thousand pounds is the thirteenth rule in the morning then I seized my thread and followed it until I was merely going
